In [1]:
import utils
from skrub import TableReport
from pathlib import Path
import pandas as pd

In [2]:
data = pd.read_parquet("data/train.parquet")
data = data.sort_values(["date", "counter_name"])
data_test = pd.read_parquet("data/final_test.parquet")
data_test = data_test.sort_values(["date", "counter_name"])

In [3]:
data = utils._merge_external_data(data)
data_test = utils._merge_external_data(data_test)

data = utils._column_rename(data)
data_test = utils._column_rename(data_test)

/Users/felix/Python_Data_Challenge-1/utils.py:76: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  date_range = pd.date_range(start=external_conditions['date'].min(), end=external_conditions['date'].max(), freq='H')
/Users/felix/Python_Data_Challenge-1/utils.py:45: FutureWarning: The behavior of Series.argmax/argmin with skipna=False and NAs, or with all-NAs is deprecated. In a future version this will raise ValueError.
  closest_value = non_nan_values.iloc[(non_nan_values - value).abs().argmin()]
/Users/felix/Python_Data_Challenge-1/utils.py:76: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  date_range = pd.date_range(start=external_conditions['date'].min(), end=external_conditions['date'].max(), freq='H')
/Users/felix/Python_Data_Challenge-1/utils.py:45: FutureWarning: The behavior of Series.argmax/argmin with skipna=False and NAs, or with all-NAs is deprecated. In a future version

In [ ]:
data = utils._process_datetime_features(data)
data_test = utils._process_datetime_features(data_test)

data, data_test = utils._add_construction_work(data, data_test)

In [ ]:
columns_to_drop = utils.columns_to_drop
df = data.drop(columns=columns_to_drop)
df_test = data_test.drop(columns=columns_to_drop)

## Previous tests

In [2]:
X_train, y_train = utils.get_train_data()
train_pipeline = utils._get_estimator()
train_pipeline.named_steps["function_transformers"].steps[0][1].func = lambda X: utils._merge_external_data(X, is_train=True, columns_to_drop_store=utils.dropped_columns_store)
train_pipeline.fit(X_train, y_train)


Pipeline(steps=[('function_transformers',
                 Pipeline(steps=[('external_data',
                                  FunctionTransformer(func=<function <lambda> at 0x10efa8670>)),
                                 ('date_features',
                                  FunctionTransformer(func=<function _process_datetime_features at 0x136ec5d80>))])),
                ('column_transformers',
                 ColumnTransformer(transformers=[('date',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['year', 'month', 'w...
                              feature_types=None, gamma=None, grow_policy=None,
                              importance_type=None,
                              interaction_constraints=None,
                              learning_rate=0.11986932069472364, max_bin=None,
                              max_cat_threshold=None, max_cat_to_onehot=None,
                              max_delta_step=None, max_depth=8, max_leaves=None,
                              min_child_weight=None, missing=nan,
                              monotone_constraints=None, multi_strategy=None,
                              n_estimators=374, n_jobs=None,
                              num_parallel_tree=None, random_state=None, ...))])

In [3]:
accuracy = train_pipeline.score(X_train, y_train)
print(f"Accuracy: {accuracy}")

Accuracy: 0.835479628986916


In [ ]:
# # Access the Ridge model
# ridge_model = train_pipeline.named_steps["model"]

# # Get the coefficients from the trained model
# coefficients = ridge_model.coef_

# # Feature names depend on the column transformer preprocessing
# column_transformer = train_pipeline.named_steps["column_transformers"]

# # Get the names of the one-hot-encoded features
# date_feature_names = column_transformer.transformers_[0][1].get_feature_names_out()
# cat_feature_names = column_transformer.transformers_[1][1].get_feature_names_out()
# num_feature_names = column_transformer.transformers_[2][1].get_feature_names_out()

# # Combine feature names into a single list
# feature_names = list(date_feature_names) + list(cat_feature_names) + list(num_feature_names)

# # Create a mapping of feature names to coefficients
# feature_importance = dict(zip(feature_names, coefficients))

# # Print feature importance
# for feature, importance in feature_importance.items():
#     print(f"{feature}: {importance}")

In [4]:
test_data = utils.get_test_data()
test_pipeline = utils._get_estimator()
test_pipeline.named_steps["function_transformers"].steps[0][1].func = lambda X: utils._merge_external_data(X, is_train=False, columns_to_drop_store=utils.dropped_columns_store)
test_data = utils.get_test_data()
y_pred = train_pipeline.predict(test_data)


In [ ]:
y_pred

In [5]:
df_submission = pd.DataFrame(y_pred, columns=["log_bike_count"])
df_submission.index = test_data.index
df_submission.index.name = "Id"

In [ ]:
df_submission

In [6]:
df_submission.to_csv("/Users/felix/Documents/X/Cours Python/Kaggle/submission/test_pipeline.csv", index=True)